In [1]:
# General Libraries
import re
import time
import warnings
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from collections import Counter
warnings.filterwarnings("ignore")

# Visualizations
import seaborn as sns
from termcolor import colored
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import (train_test_split, GridSearchCV,
                                     StratifiedKFold)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (RandomForestClassifier,  
                              GradientBoostingClassifier,ExtraTreesClassifier)
from sklearn.model_selection import cross_validate
from sklearn.metrics import (ConfusionMatrixDisplay, precision_score, recall_score)


from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,roc_curve, auc, confusion_matrix, classification_report,\
    ConfusionMatrixDisplay

from sklearn.preprocessing import LabelEncoder

import xgboost as xgb
#from lightgbm import LGBMClassifier
#from getml.predictors import XGBoostClassifier
import pickle

from sklearn.preprocessing import StandardScaler #RobustScaler

'''
# Imblearn
from imblearn.pipeline import Pipeline, make_pipeline
from imblearn.over_sampling import (SMOTE, ADASYN, BorderlineSMOTE, SVMSMOTE)
from imblearn.under_sampling import (TomekLinks, NearMiss, AllKNN,
                                     EditedNearestNeighbours, 
                                     RepeatedEditedNearestNeighbours) 
from imblearn.combine import SMOTETomek, SMOTEENN

# SHAP
import shap
'''
# import shap
# shap.initjs()


'\n# Imblearn\nfrom imblearn.pipeline import Pipeline, make_pipeline\nfrom imblearn.over_sampling import (SMOTE, ADASYN, BorderlineSMOTE, SVMSMOTE)\nfrom imblearn.under_sampling import (TomekLinks, NearMiss, AllKNN,\n                                     EditedNearestNeighbours, \n                                     RepeatedEditedNearestNeighbours) \nfrom imblearn.combine import SMOTETomek, SMOTEENN\n\n# SHAP\nimport shap\n'

In [2]:
#df = pd.read_csv('../data/full_fraud_dataset.csv', nrows=100000)
df = pd.read_csv('data/group4.csv', nrows=100000)


In [3]:
df.nunique()

ssn               96
cc_num            95
first             69
last              81
gender             1
street            96
city              56
state              1
zip               57
lat               57
long              53
city_pop          57
job               90
dob               96
acct_num          96
trans_num     100000
trans_date       706
trans_time     58676
unix_time      99910
category          14
amt            19964
is_fraud           2
merchant         201
merch_lat      99433
merch_long     99040
dtype: int64

In [4]:
df=df[["acct_num","city","trans_date","trans_time","category","amt","is_fraud","lat","long","merchant","merch_lat","merch_long"]]
#df=df[["acct_num","trans_date","trans_time","category","amt","is_fraud","lat","long","merchant","merch_lat","merch_long"]]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   acct_num    100000 non-null  float64
 1   city        100000 non-null  object 
 2   trans_date  100000 non-null  object 
 3   trans_time  100000 non-null  object 
 4   category    100000 non-null  object 
 5   amt         100000 non-null  float64
 6   is_fraud    100000 non-null  int64  
 7   lat         100000 non-null  float64
 8   long        100000 non-null  float64
 9   merchant    100000 non-null  object 
 10  merch_lat   100000 non-null  float64
 11  merch_long  100000 non-null  float64
dtypes: float64(6), int64(1), object(5)
memory usage: 9.2+ MB


# Categorical Encoding

In [5]:
df1=df.copy()
#df1['cc_num']=df1['cc_num'].astype('object')
df1['acct_num']=df1['acct_num'].astype('object')
df1['merchant']=df1['merchant'].astype('object')

df1['trans_datetime'] = pd.to_datetime(df1['trans_date'])
df1['trans_year'] = df1['trans_datetime'].dt.year.astype('object')
df1['trans_month'] = df1['trans_datetime'].dt.month.astype('object')
df1['trans_day'] = df1['trans_datetime'].dt.day.astype('object')

# transaction time
df1['trans_hour'] = df1['trans_time'].str[:2].astype('object')
df1['trans_minute'] = df1['trans_time'].str[3:5].astype('object')


df1[["trans_date","trans_time","trans_datetime","trans_year","trans_month","trans_day","trans_hour","trans_minute"]]
df1.drop(["trans_datetime","trans_date","trans_time"], inplace=True, axis=1)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   acct_num      100000 non-null  object 
 1   city          100000 non-null  object 
 2   category      100000 non-null  object 
 3   amt           100000 non-null  float64
 4   is_fraud      100000 non-null  int64  
 5   lat           100000 non-null  float64
 6   long          100000 non-null  float64
 7   merchant      100000 non-null  object 
 8   merch_lat     100000 non-null  float64
 9   merch_long    100000 non-null  float64
 10  trans_year    100000 non-null  object 
 11  trans_month   100000 non-null  object 
 12  trans_day     100000 non-null  object 
 13  trans_hour    100000 non-null  object 
 14  trans_minute  100000 non-null  object 
dtypes: float64(5), int64(1), object(9)
memory usage: 11.4+ MB


In [6]:
to_drop3 = []
for col in df1.columns:
    print(col)
    if df1[col].dtype == 'O':
        dummies = pd.get_dummies(df1[col], prefix=col, drop_first=False)
        df1 = pd.concat([df1, dummies], axis=1)
        to_drop3.append(col)
df2 = df1.drop(to_drop3, axis=1)
df2.head()


acct_num
city
category
amt
is_fraud
lat
long
merchant
merch_lat
merch_long
trans_year
trans_month
trans_day
trans_hour
trans_minute


,amt,is_fraud,lat,long,merch_lat,merch_long,acct_num_10065874467.0,acct_num_32481239072.0,acct_num_60640420337.0,acct_num_60769877980.0,...,trans_minute_55,trans_minute_56,trans_minute_57,trans_minute_58,trans_minute_59,trans_minute_5:,trans_minute_6:,trans_minute_7:,trans_minute_8:,trans_minute_9:
0,1.75,0,14.6833,120.5333,14.442359,120.658406,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,29.75,0,14.6833,120.5333,14.195802,121.075611,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,120.24,0,14.6833,120.5333,15.639445,120.936560,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,6.52,0,14.6833,120.5333,14.680228,119.877899,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,1.08,0,14.6833,120.5333,15.673455,121.303788,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


# Continuous Scaling

In [7]:
#'''
df2=df2[df2.amt<1300]
scaler = StandardScaler()
cols=["amt","lat","long","merch_lat","merch_long",]
df2[cols] = scaler.fit_transform(df2[cols])
df2
#'''

,amt,is_fraud,lat,long,merch_lat,merch_long,acct_num_10065874467.0,acct_num_32481239072.0,acct_num_60640420337.0,acct_num_60769877980.0,...,trans_minute_55,trans_minute_56,trans_minute_57,trans_minute_58,trans_minute_59,trans_minute_5:,trans_minute_6:,trans_minute_7:,trans_minute_8:,trans_minute_9:
0,-0.621339,0,0.598072,-0.945949,0.515638,-0.836872,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,-0.352168,0,0.598072,-0.945949,0.441530,-0.620026,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0.517735,0,0.598072,-0.945949,0.875447,-0.692299,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,-0.575484,0,0.598072,-0.945949,0.587134,-1.242549,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,-0.627780,0,0.598072,-0.945949,0.885669,-0.501429,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.738360,0,0.704858,-0.864209,0.755963,-1.186076,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99996,0.254717,0,0.704858,-0.864209,0.654757,-1.322147,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99997,0.641554,0,0.704858,-0.864209,0.683627,-1.148969,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99998,0.202998,0,0.704858,-0.864209,0.714156,-1.278169,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df2.to_csv('data/df_impt_cols.csv', index=None)

# AutoML FI

In [9]:
'''
# read python dict back from the file
pkl_file = open(r'pickles/automl_fi.pkl', 'rb')
automl_fi = pickle.load(pkl_file)
pkl_file.close()


automl_fi=automl_fi[["pipeline_name","validation_score","standard_deviation_cv_score","dropped_column"]].sort_values(by="validation_score",ascending=True)
automl_fi["diff"]=0.904-automl_fi["validation_score"]
automl_fi.sort_values(by="diff",ascending=False,inplace=True)
automl_fi
'''

'\n# read python dict back from the file\npkl_file = open(r\'pickles/automl_fi.pkl\', \'rb\')\nautoml_fi = pickle.load(pkl_file)\npkl_file.close()\n\n\nautoml_fi=automl_fi[["pipeline_name","validation_score","standard_deviation_cv_score","dropped_column"]].sort_values(by="validation_score",ascending=True)\nautoml_fi["diff"]=0.904-automl_fi["validation_score"]\nautoml_fi.sort_values(by="diff",ascending=False,inplace=True)\nautoml_fi\n'